In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import tifffile as tiff

from sklearn.model_selection import train_test_split

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42
seed_everything(SEED)

MS_DIR = r"D:\HocTap\NCKH_ThayDoNhuTai\Challenges\data\raw\Kaggle_Prepared\train\MS"  # ✅ sửa nếu khác
print("MS_DIR exists:", os.path.isdir(MS_DIR), "| n_files:", len(os.listdir(MS_DIR)))


MS_DIR exists: True | n_files: 600


In [2]:
def label_from_filename(fname: str) -> str:
    return os.path.basename(fname).split("_")[0]

class MSFilenameDataset(Dataset):
    def __init__(self, img_dir, transform=None, exts=(".tif",".tiff"), band_idx=None):
        self.img_dir = img_dir
        self.transform = transform
        self.band_idx = band_idx

        self.files = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(exts)])
        if len(self.files) == 0:
            raise RuntimeError(f"No tif/tiff found in {img_dir}")

        labels = sorted({label_from_filename(f) for f in self.files})
        self.class_to_idx = {c:i for i,c in enumerate(labels)}
        self.y = [self.class_to_idx[label_from_filename(f)] for f in self.files]

    def __len__(self): 
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        y = self.y[idx]
        path = os.path.join(self.img_dir, fname)

        arr = tiff.imread(path)  # (H,W,C) hoặc (C,H,W)

        # đưa về (H,W,C)
        if arr.ndim == 2:
            arr = arr[..., None]
        elif arr.ndim == 3 and arr.shape[0] == 5 and arr.shape[-1] != 5:
            arr = np.transpose(arr, (1,2,0))

        arr = arr.astype(np.float32)

        # scale về [0,1] theo ảnh
        mx = arr.max()
        if mx > 0:
            arr = arr / mx

        x = torch.from_numpy(arr).permute(2,0,1)  # C,H,W (C=5)

        # ablation: chọn bands
        if self.band_idx is not None:
            x = x[self.band_idx, :, :]

        if self.transform:
            x = self.transform(x)

        return x, y


In [3]:
ds_full = MSFilenameDataset(MS_DIR, band_idx=[0,1,2,3,4])
x0, y0 = ds_full[0]
print("x shape:", x0.shape, "y:", y0)
print("per-channel mean:", x0.mean(dim=(1,2)))
print("per-channel std :", x0.std(dim=(1,2)))
print("num classes:", len(ds_full.class_to_idx), ds_full.class_to_idx)


x shape: torch.Size([5, 64, 64]) y: 0
per-channel mean: tensor([0.1485, 0.2388, 0.2814, 0.5638, 0.6811])
per-channel std : tensor([0.0401, 0.0478, 0.0843, 0.0770, 0.1029])
num classes: 3 {'Health': 0, 'Other': 1, 'Rust': 2}


In [4]:
VAL_RATIO = 0.2

idx_all = np.arange(len(ds_full))
train_idx, val_idx = train_test_split(
    idx_all,
    test_size=VAL_RATIO,
    random_state=SEED,
    stratify=ds_full.y
)

print("train size:", len(train_idx), "| val size:", len(val_idx))


train size: 480 | val size: 120


In [5]:
def make_loaders_for_case(img_dir, band_idx, train_idx, val_idx, bs=32, num_workers=0, transform=None):
    ds = MSFilenameDataset(img_dir, transform=transform, band_idx=band_idx)
    train_ds = Subset(ds, train_idx)
    val_ds   = Subset(ds, val_idx)

    train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=num_workers)
    val_loader   = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader, ds


In [6]:
from torchvision.models import resnet18

def build_resnet18(in_channels: int, num_classes: int):
    model = resnet18(weights=None)
    model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [7]:
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    ce = nn.CrossEntropyLoss()

    total_loss, total_correct, total = 0.0, 0, 0
    for x, y in loader:
        x = x.to(device)
        y = torch.as_tensor(y, device=device)

        optimizer.zero_grad()
        logits = model(x)
        loss = ce(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        total_correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)

    return total_loss / total, total_correct / total

@torch.no_grad()
def evaluate_acc(model, loader, device):
    model.eval()
    total_correct, total = 0, 0
    for x, y in loader:
        x = x.to(device)
        y = torch.as_tensor(y, device=device)
        logits = model(x)
        total_correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)
    return total_correct / total


In [8]:
def run_case(exp_name, band_idx, img_dir, train_idx, val_idx, num_classes,
             epochs=10, bs=32, lr=1e-4, num_workers=0, transform=None):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_loader, val_loader, ds = make_loaders_for_case(
        img_dir, band_idx, train_idx, val_idx, bs=bs, num_workers=num_workers, transform=transform
    )

    model = build_resnet18(in_channels=len(band_idx), num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    os.makedirs("checkpoints", exist_ok=True)
    best_path = f"checkpoints/best_{exp_name}_resnet18.pth"
    best_acc = -1.0

    for ep in range(1, epochs+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, device)
        va_acc = evaluate_acc(model, val_loader, device)

        print(f"[{exp_name}] ep {ep}/{epochs} | train_acc={tr_acc:.4f} loss={tr_loss:.4f} | val_acc={va_acc:.4f}")

        if va_acc > best_acc:
            best_acc = va_acc
            torch.save({
                "model": model.state_dict(),
                "band_idx": band_idx,
                "best_acc": best_acc,
                "exp": exp_name,
                "class_to_idx": ds.class_to_idx,
                "seed": SEED
            }, best_path)

    print(f"[{exp_name}] BEST val_acc={best_acc:.4f} saved -> {best_path}")
    return best_acc, best_path


In [9]:
ABL = {
    "MS_noNIR": [0,1,2,3],  # bỏ band 4 (giả sử band4 là NIR)
    "drop_0":   [1,2,3,4],  # bỏ band 0
}

NUM_CLASSES = len(ds_full.class_to_idx)  # tự lấy theo folder
print("NUM_CLASSES:", NUM_CLASSES)


NUM_CLASSES: 3


In [11]:
import pandas as pd
DROP = {
    "MS_full": [0,1,2,3,4],
    "drop_0":  [1,2,3,4],
    "drop_1":  [0,2,3,4],
    "drop_2":  [0,1,3,4],
    "drop_3":  [0,1,2,4],
    "drop_4":  [0,1,2,3],
}

results = []
for name, band_idx in DROP.items():
    acc, ckpt = run_case(
        exp_name=name,
        band_idx=band_idx,
        img_dir=MS_DIR,
        train_idx=train_idx,
        val_idx=val_idx,
        num_classes=NUM_CLASSES,
        epochs=10,
        bs=32,
        lr=1e-4,
        num_workers=0
    )
    results.append({
        "exp": name,
        "bands": str(band_idx),
        "n_bands": len(band_idx),
        "val_acc": acc,
        "ckpt": ckpt
    })

df = pd.DataFrame(results).sort_values("val_acc", ascending=False)
df.to_csv("checkpoints/ablation_drop_each.csv", index=False)
df


[MS_full] ep 1/10 | train_acc=0.5708 loss=0.8968 | val_acc=0.3250
[MS_full] ep 2/10 | train_acc=0.8063 loss=0.5725 | val_acc=0.4833
[MS_full] ep 3/10 | train_acc=0.8896 loss=0.3504 | val_acc=0.4750
[MS_full] ep 4/10 | train_acc=0.9313 loss=0.1963 | val_acc=0.5667
[MS_full] ep 5/10 | train_acc=0.9688 loss=0.1085 | val_acc=0.6167
[MS_full] ep 6/10 | train_acc=0.9667 loss=0.1084 | val_acc=0.5833
[MS_full] ep 7/10 | train_acc=0.9646 loss=0.0862 | val_acc=0.5500
[MS_full] ep 8/10 | train_acc=0.9771 loss=0.0875 | val_acc=0.5667
[MS_full] ep 9/10 | train_acc=0.9646 loss=0.0683 | val_acc=0.5833
[MS_full] ep 10/10 | train_acc=0.9750 loss=0.0592 | val_acc=0.5833
[MS_full] BEST val_acc=0.6167 saved -> checkpoints/best_MS_full_resnet18.pth
[drop_0] ep 1/10 | train_acc=0.5500 loss=0.9820 | val_acc=0.3000
[drop_0] ep 2/10 | train_acc=0.7125 loss=0.6516 | val_acc=0.4583
[drop_0] ep 3/10 | train_acc=0.8854 loss=0.3967 | val_acc=0.5083
[drop_0] ep 4/10 | train_acc=0.9542 loss=0.1923 | val_acc=0.5500
[d

,exp,bands,n_bands,val_acc,ckpt
4,drop_3,"[0, 1, 2, 4]",4,0.633333,checkpoints/best_drop_3_resnet18.pth
3,drop_2,"[0, 1, 3, 4]",4,0.625000,checkpoints/best_drop_2_resnet18.pth
0,MS_full,"[0, 1, 2, 3, 4]",5,0.616667,checkpoints/best_MS_full_resnet18.pth
5,drop_4,"[0, 1, 2, 3]",4,0.616667,checkpoints/best_drop_4_resnet18.pth
1,drop_0,"[1, 2, 3, 4]",4,0.608333,checkpoints/best_drop_0_resnet18.pth
2,drop_1,"[0, 2, 3, 4]",4,0.591667,checkpoints/best_drop_1_resnet18.pth


In [12]:
print("train size:", len(train_idx), "val size:", len(val_idx))
print("val_idx hash:", hash(tuple(val_idx[:200])))


train size: 480 val size: 120
val_idx hash: -5796670799930339884
